# Inference for a Generalized Langevin Equation

**Feiyu Zhu, Martin Lysy, University of Waterloo**

**February 10, 2022**

## The GLE and its Quasi-Markov Representation

Consider a generalized Langevin equation (GLE) for a position process $X(t)$ given by

$$
\ddot X(t) = - \pot'_{\pph}(X(t)) - \int_0^t \gamma(t-s) \dot X(s) \ud s + F(t),
$$

where $V(t) = \frac{\ud}{\ud t} X(t) = \dot X(t)$ is the velocity process, $\ddot X(t) = \dot V(t)$ is the acceleration process, $\pot'_{\pph}(X)$ is the derivative of the potential energy $\pot_{\pph}(X)$, and $F(t)$ is a stationary Gaussian process with autocorrelation

$$
\cov(F(t), F(t+h)) = \beta^{-1} \gamma(h) = \beta^{-1} \sum_{k=1}^K \mu_k^2 e^{-\rho_k h}.
$$

In most applications $\beta$ is known, such that the model parameters are $\tth = (\pph, \mmu, \rrh)$, where $\mu_k, \rho_k > 0$.  The goal is to estimate these parameters from discrete observations $\XX = (X(t_0), \ldots, X(t_N))$.  To do this, it is useful to note that $X(t)$ has the same distribution as the stochastic differential equation (SDE) given by

$$
\begin{aligned}
\ud X(t) &= V(t) \ud t \\
\ud V(t) &= -\pot'_{\pph}(X(t)) \ud t + \sum_{i=1}^K \mu_k Z_k(t)  \ud t \\
\ud Z_k(t) &= -\left(\mu_k V(t) + \rho_k Z_k(t) \right) \ud t + \sigma_k \ud B_k(t),
\end{aligned}
$$

where $\sigma_k = \sqrt{2\rho_k}/\beta$.  Moreover, let $\ZZ(t) = (Z_1(t), \ldots, Z_K(t))$ and $\WW(t) = (X(t), V(t), \ZZ(t))$.  Then the stationary distribution of $\WW(t)$ is [TBD].


## Modified Euler Discretization Scheme

Let $X_n = X(t_n)$, $V_n = V(t_n)$, $\ZZ_n = \ZZ(t_n)$, etc., and let $\Delta X_n(s) = X(t_n+s) - X(t_n)$, and similarly for $\Delta V_n(s)$, $\Delta \WW_n(s)$, $\Delta B_{nk}(s)$, etc.  Then for small $s$, the SDE above may be approximated on the time interval $t \in (t_n, t_n+s)$ by

$$
\begin{aligned}
\Delta X_n(s)    & = \int_{0}^{s} V_n + \Delta V_n(h) \ud h \\
\Delta V_n(s)    
%                 & = \int_{t_n}^{t_n+\dt} \left[-U'_{\pph}(X(t)) + \sum_{i=1}^K \mu_k Z_k(t)\right]  \ud t \\
                 & = -\pot'_{\pph}(X_n) s + \sum_{i=1}^K \mu_k \int_{0}^{s} Z_{nk} + \Delta Z_{nk}(h) \ud h \\
\Delta Z_{nk}(s) 
%                 & = - \int_{t_n}^{t_n+1} \left(\mu_k V(t) + \rho_k Z_k(t) \right) \ud t + \sqrt{2\rho_k}/\beta \int_{t_n}^{t_n+\dt} \ud B_k(t) \\
                 & = -\left(\mu_k V_n + \rho_k Z_{nk}\right)s + \sigma_k \Delta B_{nk}(s).
\end{aligned}
$$
 

Let $U_{nk}^{(j)}(s)$ for $j \in \{0,1,2\}$ be defined as

$$
\begin{aligned}
U^{(2)}_{nk}(s) & = \sigma_k \Delta B_{nk}(s), \\
U^{(1)}_{nk}(s) & = \int_0^s U^{(2)}_{nk}(h) \ud h, \\
U^{(0)}_{nk}(s) & = \int_0^s U^{(1)}_{nk}(h) \ud h.
\end{aligned}
$$

Let $\UU_n^{(j)}(t) = (U^{(j)}_{n1}(t), \ldots, U^{(j)}_{nK}(t))$ and consider the $3K$-dimensional process $\UU_n(t) = (\UU^{(0)}_n(t), \UU^{(1)}_n(t), \UU^{(2)}_n(t))$.  Then upon letting $A_{nk} = \mu_k V_n + \rho_k Z_{nk}$ and $\AA_{n} = (A_{n1}, \ldots, A_{nK})$, we have

$$
\Delta \WW_n(s) = \lla(\WW_n, s) + \tilde \ZZ_n(s),
$$

where each of the three terms above is a $(K+2)$-dimensional process with 

$$
\begin{aligned}
\lla(\WW_n, s) & = 
\begin{bmatrix}
V_n -\tfrac 1 2 \pot'_{\pph}(X_n) s^2 + \mmu'\left(\tfrac 1 2 \ZZ_n s^2 - \tfrac 1 6 \AA_n s^3\right) \\
-\pot'_{\pph}(X_n) s + \mmu' \left(\ZZ_n s - \tfrac 1 2 \AA_n s^2 \right) \\
-\AA_n s
\end{bmatrix}, & 
\tilde \ZZ_n(s) & = 
\begin{bmatrix} 
\mmu' & \bz & \bz \\
\bz & \mmu' & \bz \\
\bz & \bz & \Id_{K\times K}
\end{bmatrix}\UU_n(s).
\end{aligned}
$$

Note that $\UU_n(s)$ is a Markov process, with $\UU_n(0) = \bz$ and

$$
\UU_n(s+h) \mid \UU_n(s) \sim \N\left\{\left(\RR(h) \otimes \Id_{K\times K}\right) \UU_n(s), \SSi(h) \otimes \diag(\ssi^2) \right\},
$$

where 

$$
\begin{aligned}
\RR(h) & = 
\begin{bmatrix}
1 & h & \tfrac 1 2 h^2 \\
0 & 1 & h \\
0 & 0 & 1
\end{bmatrix}, & 
\SSi(h) & =
\begin{bmatrix}
\tfrac{1}{20} h^5 & \tfrac{1}{8} h^4 & \tfrac{1}{6} h^3 \\
\tfrac{1}{8} h^4 & \tfrac{1}{3} h^3 & \tfrac{1}{2} h^2 \\
\tfrac{1}{6} h^3 & \tfrac{1}{2} h^2 & h
\end{bmatrix},
\end{aligned}
$$

and where $\otimes$ is the Kronecker matrix product.  It follows that $\tilde \ZZ_n(s)$ is also a Markov process with $\tilde \ZZ_n(0) = \bz$ and 

$$
\tilde \ZZ_n(s+h) \mid \tilde \ZZ_n(s) \sim 
\N\left( \tilde \RR(h) \tilde \ZZ_n(s), \tilde \SSi(h) \right),
$$

where

$$
\begin{aligned}
\tilde \RR(h) & = \begin{bmatrix}
1 & h & \tfrac 1 2 h^2 \mmu' \\
0 & 1 & h \mmu' \\
0 & 0 & \Id_{K\times K}
\end{bmatrix}, & 
\tilde \SSi(h) & = \begin{bmatrix} 
\tfrac{1}{20} \gamma s^5 & \tfrac {1}{8} \gamma s^4 & \tfrac{1}{6}s^3 \mmu' \diag(\ssi^2) \\
\tfrac {1}{8} \gamma s^4 & \tfrac{1}{3} \gamma s^3 & \tfrac{1}{2}s^2 \mmu' \diag(\ssi^2) \\
\tfrac{1}{6}s^3 \diag(\ssi^2) \mmu & \tfrac{1}{2}s^2 \diag(\ssi^2) \mmu  & s \diag(\ssi^2)
\end{bmatrix},
\end{aligned}
$$

and where $\gamma = \sum_{i=1}^K \mu_k^2 \sigma_k^2$.  Thus, let $t_n = n \dt$.  Then we have the following algorithm to simulate $\WW_0, \ldots, \WW_N$, $\WW_n = \WW(t_n) = \WW(n \dt)$:

- Fix the value of $\WW_0$, or draw it from its stationary distribution.

- Given $\WW_n$, generate $\WW_{n+1}$ via

    $$
    \WW_{n+1} = \WW_n + \lla(\WW_n, \dt) + \tilde \ZZ_n(\dt),
    $$
    
    where $\tilde \ZZ_n(\dt) \iid \N(\bz, \tilde \SSi(\dt))$.

## Particle Filtering

As described in the SDE document, the setting for parameter inference is via noisy GLE observations $\YY = (\YY_0, \ldots, \YY_N)$

$$
\YY_n \ind \N(\AA \WW(t_n), \OOm),
$$

where for simplicity we assume that $t_n = n \dt$.  In fact, we are most interested in the case where $\AA = \left[\begin{smallmatrix} 1 & \bz \\ \bz & \bz \end{smallmatrix}\right]$ and $\OOm = \varepsilon \AA$, which corresponds to observing $Y_n \ind \N(X_n, \varepsilon^2)$, and letting $\varepsilon \to 0$.

For resolution number $m \ge 1$, let $\WW\up m_n$ denote the value of the SDE at time $t = n \dt/m$, such that $\WW\up m_{mn} = \WW_n = \WW(t_n)$.  Then in the PFJAX notation we have

$$
\begin{aligned}
\xx_n & = \WW\up m_{(n-1)m+1:tm}, & \yy_n & = \YY_n.
\end{aligned}
$$

### Bridge Proposal

*[TBD]* A bridge proposal for $\xx_n$ is constructed as follows:

- Suppose for simplicity that $n=1$, that $\WW\up m_{(n-1)m+i} = \WW\up m_{i} = \WW(i \dt/m)$ is given and we wish to draw the proposal for $\WW\up m_{i+1} = \WW((i+1)\dt/m)$.

- For $s \ge 0$, let $\WW_i(s) = \WW(i\dt_m + s)$. Then using the logic above, consider the approximation

    $$
    \WW(i\dt/m + s) = \mmu_i(s) + \tilde \ZZ_i(s), 
    $$
    
    where $\mmu_i(s) = \WW\up m_i + \lla(\WW\up m_i, s)$ and $\tilde \ZZ_i(s)$ is a Markov process as defined above.  Under this approximation we have the model
    
    $$
    \begin{aligned}
    \WW\up m_{i+1} \mid \WW\up m_i & \sim \N(\mmu_i(\dt/m), \tilde{\SSi}(\dt/m)) \\
    \WW\up m_{m} \mid \WW\up m_{i+1}, \WW\up m_i & \sim \N(\mmu_i(\dt) + \tilde{\RR}(\dt_i)(\WW\up m_{i+1} - \mmu_i(\dt/m)), \tilde{\SSi}(\dt_i)) \\
    \YY_1 \mid \WW\up m_{m}, \WW\up m_{i+1}, \WW\up m_i & \sim \N(\AA \WW\up m_{m}, \OOm),
    \end{aligned}
    $$
    
    where $\dt_i = (m-i-1)\dt/m$.  We may now use standard formulas for joint and conditional distributions of the multivariate normal to sample $\WW\up m_{i+1} \sim p(\WW\up m_{i+1} \mid \WW\up m_i, \YY_1)$, namely
    
    $$
    \WW\up m_{i+1} \mid \WW\up m_i, \YY_1 \sim \N(\lla_{i+1}, \OOm_{i+1}),
    $$
    
    where the formulas for $\lla_{i+1}$ and $\OOm_{i+1}$ are derived in a more general context in `bridge_proposal.ipynb`.